In [1]:
import math
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn import preprocessing

import numpy as np
import pandas as pd

import time
import random
import matplotlib.pyplot as plt
from scipy import interp
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from tqdm import tqdm

In [2]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report

In [3]:
def load_data(directory):
    D_SSM1 = np.loadtxt(directory + '/D_SSM1.txt')
    D_SSM2 = np.loadtxt(directory + '/D_SSM2.txt')
    D_GSM = np.loadtxt(directory + '/D_GSM.txt')
    M_FSM = np.loadtxt(directory + '/M_FSM.txt')
    M_GSM = np.loadtxt(directory + '/M_GSM.txt')
    D_SSM = (D_SSM1 + D_SSM2) / 2

    ID = np.zeros(shape=(D_SSM.shape[0], D_SSM.shape[1]))
    IM = np.zeros(shape=(M_FSM.shape[0], M_FSM.shape[1]))
    for i in range(D_SSM.shape[0]):
        for j in range(D_SSM.shape[1]):
            if D_SSM[i][j] == 0:
                ID[i][j] = D_GSM[i][j]
            else:
                ID[i][j] = D_SSM[i][j]
    for i in range(M_FSM.shape[0]):
        for j in range(M_FSM.shape[1]):
            if M_FSM[i][j] == 0:
                IM[i][j] = M_GSM[i][j]
            else:
                IM[i][j] = M_FSM[i][j]
                
    ID = pd.DataFrame(ID).reset_index()
    IM = pd.DataFrame(IM).reset_index()
    ID.rename(columns = {'index':'id'}, inplace = True)
    IM.rename(columns = {'index':'id'}, inplace = True)
    ID['id'] = ID['id'] + 1
    IM['id'] = IM['id'] + 1
    
    return ID, IM

def sample(directory, random_seed):
    all_associations = pd.read_csv(directory + '/all_mirna_disease_pairs.csv', names=['miRNA', 'disease', 'label'])
    known_associations = all_associations.loc[all_associations['label'] == 1]
    unknown_associations = all_associations.loc[all_associations['label'] == 0]
    random_negative = unknown_associations.sample(n=known_associations.shape[0], random_state=random_seed, axis=0)

    sample_df = known_associations.append(random_negative)
    sample_df.reset_index(drop=True, inplace=True)

    return sample_df

In [4]:
def performances(y_true, y_pred, y_prob):

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()

    pos_acc = tp / sum(y_true)
    neg_acc = tn / (len(y_pred) - sum(y_pred)) # [y_true=0 & y_pred=0] / y_pred=0
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    recall = tp / (tp+fn)
    precision = tp / (tp+fp)
    f1 = 2*precision*recall / (precision+recall)
    
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
    print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
    print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
    print('acc={:.4f}|precision={:.4f}|recall={:.4f}|f1={:.4f}|auc={:.4f}|aupr={:.4f}|pos_acc={:.4f}|neg_acc={:.4f}'.format(accuracy, precision, recall, f1, roc_auc, aupr, pos_acc, neg_acc))
    return (y_true, y_pred, y_prob), (accuracy, precision, recall, f1, roc_auc, aupr, pos_acc, neg_acc)

In [5]:
def obtain_data(directory, isbalance):

    ID, IM = load_data(directory)

    if isbalance:
        dtp = sample(directory, random_seed = 1234)
    else:
        dtp = pd.read_csv(directory + '/all_mirna_disease_pairs.csv', names=['miRNA', 'disease', 'label'])

    mirna_ids = list(set(dtp['miRNA']))
    disease_ids = list(set(dtp['disease']))
    random.shuffle(mirna_ids)
    random.shuffle(disease_ids)
    print('# miRNA = {} | Disease = {}'.format(len(mirna_ids), len(disease_ids)))

    mirna_test_num = int(len(mirna_ids) / 5)
    disease_test_num = int(len(disease_ids) / 5)
    print('# Test: miRNA = {} | Disease = {}'.format(mirna_test_num, disease_test_num))    
    
    samples = pd.merge(pd.merge(dtp, ID, left_on = 'disease', right_on = 'id'), IM, left_on = 'miRNA', right_on = 'id')
    samples.drop(labels = ['id_x', 'id_y'], axis = 1, inplace = True)
    
    return ID, IM, dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples

In [6]:
def generate_task_Tp_train_test_idx(samples):
    kf = KFold(n_splits = 5, shuffle = True, random_state = 1234)

    train_index_all, test_index_all, n = [], [], 0
    train_id_all, test_id_all = [], []
    fold = 0
    for train_idx, test_idx in tqdm(kf.split(samples.iloc[:, 3:])): #train_index与test_index为下标
        print('-------Fold ', fold)
        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)

        train_id_all.append(np.array(dtp.iloc[train_idx][['miRNA', 'disease']]))
        test_id_all.append(np.array(dtp.iloc[test_idx][['miRNA', 'disease']]))

        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        fold += 1
    return train_index_all, test_index_all, train_id_all, test_id_all

In [7]:
def generate_task_Tm_Td_train_test_idx(item, ids, dtp):
    
    test_num = int(len(ids) / 5)
    
    train_index_all, test_index_all = [], []
    train_id_all, test_id_all = [], []
    
    for fold in range(5):
        print('-------Fold ', fold)
        if fold != 4:
            test_ids = ids[fold * test_num : (fold + 1) * test_num]
        else:
            test_ids = ids[fold * test_num :]

        train_ids = list(set(ids) ^ set(test_ids))
        print('# {}: Train = {} | Test = {}'.format(item, len(train_ids), len(test_ids)))

        test_idx = dtp[dtp[item].isin(test_ids)].index.tolist()
        train_idx = dtp[dtp[item].isin(train_ids)].index.tolist()
        random.shuffle(test_idx)
        random.shuffle(train_idx)
        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        assert len(train_idx) + len(test_idx) == len(dtp)

        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)
        
        train_id_all.append(train_ids)
        test_id_all.append(test_ids)
        
    return train_index_all, test_index_all, train_id_all, test_id_all

# 随机森林

In [8]:
def run_rf(train_index_all, test_index_all, samples):
    
    fold = 0
    for train_idx, test_idx in zip(train_index_all, test_index_all):
        print('-----------------------Fold = ', str(fold))

        X = samples.iloc[:, 3:]
        y = samples['label']

        scaler = preprocessing.MinMaxScaler().fit(X.iloc[train_idx,:])
        X = scaler.transform(X)

        x_train, y_train = X[train_idx], y[train_idx]
        x_test, y_test = X[test_idx], y[test_idx]

        clf = RandomForestClassifier(random_state = 19961231)
        clf.fit(x_train, y_train)

        y_train_prob = clf.predict_proba(x_train)
        y_test_prob = clf.predict_proba(x_test)

        y_train_pred = clf.predict(x_train)
        y_test_pred = clf.predict(x_test)

        print('Train:')
        ys_train, metrics_train = performances(y_train, y_train_pred, y_train_prob[:, 1])
        print('Test:')
        ys_test, metrics_test = performances(y_test, y_test_pred, y_test_prob[:, 1])

        fold += 1
    
    return ys_train, metrics_train, ys_test, metrics_test

# DNN

In [9]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.metrics import *
from keras import callbacks
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score, auc, precision_recall_curve, confusion_matrix
import numpy as np
import sklearn.metrics as metrics
from collections import Counter

Using TensorFlow backend.


In [10]:
def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    # Calculates the F score, the weighted harmonic mean of precision and recall.

    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
        
    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def f1(y_true, y_pred):
    # Calculates the f-measure, the harmonic mean of precision and recall.
    return fbeta_score(y_true, y_pred, beta=1)


def transfer(y_pred):
    return [[0,1][x>0.5] for x in y_pred.reshape(-1)]

In [11]:
def Model(x):
    model = Sequential()
    model.add(Dense(1024, activation='elu', input_shape=(x.shape[1],)))
    model.add(Dense(512, activation='elu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(lr = 0.0001)
    model.compile(optimizer = optimizer, loss = binary_crossentropy, metrics=[binary_accuracy, f1, recall, precision])
    return model

In [12]:
def run_dnn(train_index_all, test_index_all, samples):
    
    fold = 0
    for train_idx, test_idx in zip(train_index_all, test_index_all):
        print('----------------------- Fold = ', str(fold))

        X = samples.iloc[:, 3:]
        y = samples['label']

        scaler = preprocessing.MinMaxScaler().fit(X.iloc[train_idx,:])
        X = scaler.transform(X)

        x_train, y_train = X[train_idx], y[train_idx]
        x_test, y_test = X[test_idx], y[test_idx]

        model = Model(x_train)
        early_stopping = EarlyStopping(monitor='val_loss', patience = 50)
        model.fit(x_train, y_train, epochs = 500, batch_size = 256, validation_data=(x_test, y_test), callbacks=[early_stopping],verbose = 2)

        y_train_pred, y_test_pred = model.predict(x_train, verbose = 0), model.predict(x_test, verbose = 0)
        y_train_prob, y_test_prob = model.predict_proba(x_train), model.predict_proba(x_test)

        if len(Counter(y_train_pred.reshape(-1))) > 2: 
            y_train_pred = transfer(y_train_pred)
        else:
            print(Counter(y_train_pred.reshape(-1)))
        if len(Counter(y_test_pred.reshape(-1))) > 2: 
            y_test_pred = transfer(y_test_pred)
        else:
            print(Counter(y_test_pred.reshape(-1)))

        performances_train = performances(y_train, y_train_pred, y_train_prob)
        performances_test = performances(y_test, y_test_pred, y_test_prob)

        fold += 1
    
    return y_train_pred, y_test_pred, y_train_prob, y_test_prob, performances_train, performances_test

# Run

## RF

In [13]:
directory = 'data'
for isbalance in [True, False]:
    
    ID, IM, dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples = obtain_data(directory, 
                                                                                                 isbalance)
    for task in ['Tp', 'Tm', 'Td']:
        
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tm':
            item = 'miRNA'
            ids = mirna_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        elif task == 'Td':
            item = 'disease'
            ids = disease_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        ys_train, metrics_train, ys_test, metrics_test = run_rf(train_index_all, test_index_all, samples)

# miRNA = 495 | Disease = 383
# Test: miRNA = 99 | Disease = 76


5it [00:00, 171.76it/s]

========== isbalance = True | task = Tp
-------Fold  0
# Pairs: Train = 8688 | Test = 2172
-------Fold  1
# Pairs: Train = 8688 | Test = 2172
-------Fold  2
# Pairs: Train = 8688 | Test = 2172
-------Fold  3
# Pairs: Train = 8688 | Test = 2172
-------Fold  4
# Pairs: Train = 8688 | Test = 2172
-----------------------Fold =  0


Train:
tn = 4330, fp = 0, fn = 0, tp = 4358
y_pred: 0 = 4330 | 1 = 4358
y_true: 0 = 4330 | 1 = 4358
acc=1.0000|precision=1.0000|recall=1.0000|f1=1.0000|auc=1.0000|aupr=1.0000|pos_acc=1.0000|neg_acc=1.0000
Test:
tn = 945, fp = 155, fn = 99, tp = 973
y_pred: 0 = 1044 | 1 = 1128
y_true: 0 = 1100 | 1 = 1072
acc=0.8831|precision=0.8626|recall=0.9076|f1=0.8845|auc=0.9492|aupr=0.9443|pos_acc=0.9076|neg_acc=0.9052
-----------------------Fold =  1
Train:
tn = 4336, fp = 0, fn = 0, tp = 4352
y_pred: 0 = 4336 | 1 = 4352
y_true: 0 = 4336 | 1 = 4352
acc=1.0000|precision=1.0000|recall=1.0000|f1=1.0000|auc=1.0000|aupr=1.0000|pos_acc=1.0000|neg_acc=1.0000
Test:
tn = 943, fp = 151, fn = 131, tp = 947
y_pred: 0 = 1074 | 1 = 1098
y_true: 0 = 1094 | 1 = 1078
acc=0.8702|precision=0.8625|recall=0.8785|f1=0.8704|auc=0.9396|aupr=0.9354|pos_acc=0.8785|neg_acc=0.8780
-----------------------Fold =  2
Train:
tn = 4332, fp = 1, fn = 0, tp = 4355
y_pred: 0 = 4332 | 1 = 4356
y_true: 0 = 4333 | 1 = 4355
acc=0.9999|pr

5it [00:00, 36.40it/s]

========== isbalance = False | task = Tp
-------Fold  0
# Pairs: Train = 151668 | Test = 37917
-------Fold  1
# Pairs: Train = 151668 | Test = 37917
-------Fold  2
# Pairs: Train = 151668 | Test = 37917
-------Fold  3
# Pairs: Train = 151668 | Test = 37917
-------Fold  4
# Pairs: Train = 151668 | Test = 37917
-----------------------Fold =  0


Train:
tn = 147293, fp = 0, fn = 4, tp = 4371
y_pred: 0 = 147297 | 1 = 4371
y_true: 0 = 147293 | 1 = 4375
acc=1.0000|precision=1.0000|recall=0.9991|f1=0.9995|auc=1.0000|aupr=1.0000|pos_acc=0.9991|neg_acc=1.0000
Test:
tn = 36717, fp = 145, fn = 694, tp = 361
y_pred: 0 = 37411 | 1 = 506
y_true: 0 = 36862 | 1 = 1055
acc=0.9779|precision=0.7134|recall=0.3422|f1=0.4625|auc=0.9379|aupr=0.5403|pos_acc=0.3422|neg_acc=0.9814
-----------------------Fold =  1
Train:
tn = 147266, fp = 0, fn = 5, tp = 4397
y_pred: 0 = 147271 | 1 = 4397
y_true: 0 = 147266 | 1 = 4402
acc=1.0000|precision=1.0000|recall=0.9989|f1=0.9994|auc=1.0000|aupr=1.0000|pos_acc=0.9989|neg_acc=1.0000
Test:
tn = 36726, fp = 163, fn = 637, tp = 391
y_pred: 0 = 37363 | 1 = 554
y_true: 0 = 36889 | 1 = 1028
acc=0.9789|precision=0.7058|recall=0.3804|f1=0.4943|auc=0.9366|aupr=0.5652|pos_acc=0.3804|neg_acc=0.9830
-----------------------Fold =  2
Train:
tn = 147357, fp = 0, fn = 1, tp = 4310
y_pred: 0 = 147358 | 1 = 4310
y_true: 0 = 147357

## DNN 

In [14]:
directory = 'data'
for isbalance in [True, False]:
    
    ID, IM, dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples = obtain_data(directory, 
                                                                                                 isbalance)
    for task in ['Tp', 'Tm', 'Td']:
        
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tm':
            item = 'miRNA'
            ids = mirna_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        elif task == 'Td':
            item = 'disease'
            ids = disease_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        y_train_pred, y_test_pred, y_train_prob, y_test_prob, performances_train, performances_test = run_dnn(train_index_all, test_index_all, samples)

# miRNA = 495 | Disease = 383
# Test: miRNA = 99 | Disease = 76


5it [00:00, 213.67it/s]

========== isbalance = True | task = Tp
-------Fold  0
# Pairs: Train = 8688 | Test = 2172
-------Fold  1
# Pairs: Train = 8688 | Test = 2172
-------Fold  2
# Pairs: Train = 8688 | Test = 2172
-------Fold  3
# Pairs: Train = 8688 | Test = 2172
-------Fold  4
# Pairs: Train = 8688 | Test = 2172
----------------------- Fold =  0


Train on 8688 samples, validate on 2172 samples
Epoch 1/500
1s - loss: 0.4125 - binary_accuracy: 0.8100 - f1: 0.7776 - recall: 0.7638 - precision: 0.8488 - val_loss: 0.3206 - val_binary_accuracy: 0.8600 - val_f1: 0.8108 - val_recall: 0.7966 - val_precision: 0.8450
Epoch 2/500
0s - loss: 0.3525 - binary_accuracy: 0.8460 - f1: 0.8458 - recall: 0.8473 - precision: 0.8463 - val_loss: 0.3122 - val_binary_accuracy: 0.8610 - val_f1: 0.8182 - val_recall: 0.8334 - val_precision: 0.8121
Epoch 3/500
0s - loss: 0.3486 - binary_accuracy: 0.8452 - f1: 0.8443 - recall: 0.8431 - precision: 0.8485 - val_loss: 0.3090 - val_binary_accuracy: 0.8633 - val_f1: 0.8153 - val_recall: 0.8172 - val_precision: 0.8273
Epoch 4/500
0s - loss: 0.3412 - binary_accuracy: 0.8491 - f1: 0.8490 - recall: 0.8511 - precision: 0.8489 - val_loss: 0.3127 - val_binary_accuracy: 0.8559 - val_f1: 0.8174 - val_recall: 0.8441 - val_precision: 0.8003
Epoch 5/500
0s - loss: 0.3433 - binary_accuracy: 0.8507 - f1: 0.8508 - recall: 0.856

5it [00:00, 27.49it/s]

========== isbalance = False | task = Tp
-------Fold  0
# Pairs: Train = 151668 | Test = 37917
-------Fold  1
# Pairs: Train = 151668 | Test = 37917
-------Fold  2
# Pairs: Train = 151668 | Test = 37917
-------Fold  3
# Pairs: Train = 151668 | Test = 37917
-------Fold  4
# Pairs: Train = 151668 | Test = 37917
----------------------- Fold =  0


Train on 151668 samples, validate on 37917 samples
Epoch 1/500
18s - loss: 0.0836 - binary_accuracy: 0.9728 - f1: 0.2595 - recall: 0.1874 - precision: 0.5294 - val_loss: 0.0761 - val_binary_accuracy: 0.9750 - val_f1: 0.1471 - val_recall: 0.1105 - val_precision: 0.2657
Epoch 2/500
18s - loss: 0.0792 - binary_accuracy: 0.9748 - f1: 0.3259 - recall: 0.2397 - precision: 0.6152 - val_loss: 0.0744 - val_binary_accuracy: 0.9755 - val_f1: 0.1110 - val_recall: 0.0765 - val_precision: 0.2868
Epoch 3/500
18s - loss: 0.0784 - binary_accuracy: 0.9749 - f1: 0.3334 - recall: 0.2482 - precision: 0.6181 - val_loss: 0.0765 - val_binary_accuracy: 0.9756 - val_f1: 0.1156 - val_recall: 0.0800 - val_precision: 0.2759
Epoch 4/500
18s - loss: 0.0779 - binary_accuracy: 0.9752 - f1: 0.3411 - recall: 0.2507 - precision: 0.6427 - val_loss: 0.0775 - val_binary_accuracy: 0.9747 - val_f1: 0.2577 - val_recall: 0.2397 - val_precision: 0.3379
Epoch 5/500
14s - loss: 0.0769 - binary_accuracy: 0.9756 - f1: 0.3627 - recal